In [1]:
#Inporatamos la libreria pandas
import pandas as pd 
#Inporatamos la libreria seaborn
import seaborn as sns
#Inporatamos la libreria numpy
import numpy as np

In [ ]:
#El mismo pandas tiene diferntes tipo de archivos para cargar el dataset
df = pd.read_csv('titanic.csv')
df.head()